# Install the dependencies

python3 -m pip install --break-system-packages --user requests tqdm pygltflib

### Step 1 - Get model sizes & path

Option 1 - Extract manually:
1. Run "git clone https://huggingface.co/datasets/allenai/objaverse" and then abort the command when it starts to download the models.
2. This will create a git repo folder, you then can run "python dump_gitcommits.py > out.txt" to dump the entire commit history
3. Then you call extract_models_from_dump("out.txt") to parse and get all the model paths and their sizes.

Option 2 - Use the pre-extracted json (model_sizes.json.gz)

In [29]:
import json 
import gzip

def extract_models_from_dump(file_path):
    model_sizes = {}
    current_model = None
    with open(file_path, 'r') as file:
        for line in file:
            # Get model path
            if ".glb" in line:
                # Extract model path
                model_path = line.split()[-1].strip()
                model_path = model_path.replace("b/", "")
                current_model = model_path
            # Get current_model size
            elif current_model and "size" in line: 
                
                size = int(line.split()[-1].strip()) 
                model_sizes[current_model] = size 
                current_model = None
    return model_sizes
 
 
 ## Option 1
#model_sizes = extract_models_from_dump("out.txt")  


## Option 2
with gzip.open("model_sizes.json.gz", 'rb') as gzip_file: 
    model_sizes = json.loads(gzip_file.read().decode('utf-8'))
    
print(len(model_sizes))

798759


### Download the meshes as per specified size limit

In [30]:
import os
import requests
from tqdm import tqdm  
from concurrent.futures import ThreadPoolExecutor 

def download_model(model_url, save_path):
    try:
        response = requests.get(model_url)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
                #print(f"Downloaded: {save_path}")
        else:
            print(f"Failed to download: {model_url}")
    except Exception as e:
        print(f"Error downloading: {model_url}, {e}")

def download_filtered_models(model_sizes, base_url, save_dir, minKb, maxKb, num_threads = 6, maxDownloadedMeshes = 250000):
    filtered_models = {model_path: size for model_path, size in model_sizes.items() if minKb < size < maxKb * 1024}
    
    downloaded_meshes = 0

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        for model_path, size in filtered_models.items():
            if downloaded_meshes >= maxDownloadedMeshes:
                break

            folder_name = os.path.dirname(model_path)
            sub_folder = os.path.join(save_dir, folder_name)
            os.makedirs(sub_folder, exist_ok=True)
            
            file_name = os.path.basename(model_path)
            save_path = os.path.join(sub_folder, file_name)
            
            if not os.path.exists(save_path):
                model_url = f"{base_url}/{model_path}?download=true"
                futures.append(executor.submit(download_model, model_url, save_path))
                
                downloaded_meshes += 1
                
        for future in tqdm(futures, total=len(futures)):
            future.result()
            
base_url = "https://huggingface.co/datasets/allenai/objaverse/resolve/main"  
save_dir = f'./objaverse' 

os.makedirs(save_dir, exist_ok=True)   
download_filtered_models(model_sizes, base_url, save_dir, minKb = 10, maxKb = 200, num_threads = 6, maxDownloadedMeshes = 10) 

100%|██████████| 10/10 [00:02<00:00,  4.32it/s]


### Download metadata

In [31]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
 
def download_file(url, folder_path, filename):
    url = url + "?download=true"
    print(url)
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # If the response was successful, no Exception will be raised
        with open(os.path.join(folder_path, filename), 'wb') as f:
            f.write(response.content) 
        return True
    except Exception as err:
        print(f"Failed to download {filename}. Error: {err}")
        return False

def download_metadata(base_url, save_dir, num_threads=6):
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        for i in range(1, 161):
            filename = f"000-{i:03d}.json.gz"
            file_url = base_url + filename
            futures.append(executor.submit(download_file, file_url, save_dir, filename))
         
        for future in tqdm(futures, total=len(futures)):
            result = future.result()
            if not result:
                continue
            
base_url = "https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/" 
save_dir = './objaverse/metadata'
os.makedirs(save_dir, exist_ok=True)   

download_metadata(base_url, save_dir)

https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-001.json.gz?download=truehttps://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-002.json.gz?download=true

https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-003.json.gz?download=true
https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-004.json.gz?download=true
https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-005.json.gz?download=true
https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-006.json.gz?download=true


  1%|          | 1/160 [00:00<01:58,  1.34it/s]

https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-007.json.gz?download=true
https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-008.json.gz?download=true
https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-009.json.gz?download=true
https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-010.json.gz?download=true


  3%|▎         | 5/160 [00:00<00:22,  6.87it/s]

https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-011.json.gz?download=true
https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-012.json.gz?download=true
https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-013.json.gz?download=true


...

https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-155.json.gz?download=true
https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-156.json.gz?download=true
https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-157.json.gz?download=true


 95%|█████████▌| 152/160 [00:16<00:00,  9.71it/s]

https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-158.json.gz?download=true
https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-159.json.gz?download=true
https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-160.json.gz?download=true


100%|██████████| 160/160 [00:18<00:00,  8.62it/s]

Failed to download 000-160.json.gz. Error: 404 Client Error: Not Found for url: https://huggingface.co/datasets/allenai/objaverse/resolve/main/metadata/000-160.json.gz?download=true


### Extract the metadata to a JSON with only the relevant information, e.g the models you downloaded

In [32]:
import os
import glob
import gzip
import json
from pygltflib import GLTF2, BufferFormat

# Load metadata
metadata = {}
filtered_metadata = {}
metadata_path = './objaverse/metadata'
for file_name in os.listdir(metadata_path):
    if file_name.endswith(".gz"):
        file_path = os.path.join(metadata_path, file_name)
        with gzip.open(file_path, 'rt', encoding='utf-8') as f:
            metadata = json.load(f)

input_directory = './objaverse/glbs'
output_directory = './objaverse/gltf_embedded'

os.makedirs(output_directory, exist_ok=True)

existing_models = {}
for file_path in glob.iglob(input_directory + '/**/*', recursive=True):
    if os.path.isfile(file_path):
        file_name, file_extension = os.path.splitext(file_path)
        existing_models[os.path.basename(file_name)] = file_path

        if file_extension.lower() == ".glb" and os.path.basename(file_name) in metadata:
            gltf_embedded = GLTF2().load(file_path) 
            gltf_embedded.convert_buffers(BufferFormat.DATAURI)
            gltf_file_path = os.path.join(output_directory, os.path.basename(file_name) + ".gltf")
            gltf_embedded.save(gltf_file_path)
            data = metadata[os.path.basename(file_name)]
            if data["license"] != "by":
                continue
            filtered_metadata = {
                "@context": {
                    "dc": "http://purl.org/dc/elements/1.1/"
                },
                "@id": data["uid"],
                "dc:title": data["name"],
                "dc:creator": {
                    "@id": data["user"]["uid"],
                    "dc:name": data["user"]["username"]
                },
                "dc:description": data["description"],
                "dc:date": data["createdAt"],
                "dc:identifier": data["uri"],
                "dc:source": data["viewerUrl"],
                "dc:language": "en",
                "dc:rights": data["license"],
                "dc:subject": data["tags"],
                "dc:type": "3D Model",
                "dc:relation": data["user"]["profileUrl"],
                "dc:coverage": "",
                "dc:contributor": []
            }

            with open(gltf_file_path, 'r') as f:
                gltf_json = json.load(f)

            xmp_extension = {
                "KHR_xmp_json_ld": {
                    "packets": [filtered_metadata]
                }
            }

            if 'extensions' in gltf_json['asset']:
                if 'KHR_xmp_json_ld' in gltf_json['asset']['extensions']:
                    gltf_json['asset']['extensions']['KHR_xmp_json_ld']['packets'].append(filtered_metadata)
                else:
                    gltf_json['asset']['extensions'].update(xmp_extension)
            else:
                gltf_json['asset']['extensions'] = xmp_extension

            gltf_json['asset']['extensions']['KHR_xmp_json_ld']['packet'] = len(gltf_json['asset']['extensions']['KHR_xmp_json_ld']['packets']) - 1

            if 'extensionsUsed' in gltf_json:
                if "KHR_xmp_json_ld" not in gltf_json['extensionsUsed']:
                    gltf_json['extensionsUsed'].append("KHR_xmp_json_ld")
            else:
                gltf_json['extensionsUsed'] = ["KHR_xmp_json_ld"]

            with open(gltf_file_path, 'w') as f:
                json.dump(gltf_json, f, indent=4)


python3 -m pip install --break-system-packages --user requests tqdm pygltflib

python3 -m pip install --break-system-packages --user requests tqdm pygltflib

python3 -m pip install --break-system-packages --user requests tqdm pygltflib